In [1]:
import os
import copy
import dota2api
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [2]:
API_KEY = '2029A39ECAAB627B5F71CF4C3E30F72D'
api = dota2api.Initialise(API_KEY)

In [3]:
data = pd.read_csv('../data/draft/capmodedata.csv')

In [4]:
data.head(5)

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
0,1928898739,True,2885,False,0,73,False,1,69,False,...,41.0,False,0.0,81.0,True,1.0,105.0,True,0.0,94.0
1,1928932285,True,1894,False,0,69,False,1,102,False,...,40.0,False,0.0,28.0,True,1.0,97.0,True,0.0,62.0
2,1928903165,True,2424,False,1,85,False,0,68,False,...,30.0,False,1.0,47.0,True,0.0,58.0,True,1.0,33.0
3,1928907204,True,2588,False,1,71,False,0,73,False,...,30.0,False,1.0,68.0,True,0.0,50.0,True,1.0,21.0
4,1928865084,True,3658,False,1,12,False,0,42,False,...,93.0,False,1.0,67.0,True,0.0,72.0,True,1.0,40.0


In [ ]:
data.columns

In [ ]:
# # # Getting Heroes ID according to the name
heroes = api.get_heroes()

In [ ]:
heroes_dict = {}
for i in range (len(heroes['heroes'])):
    hero_id = heroes['heroes'][i]['id']
    hero_name = heroes['heroes'][i]['localized_name']
    heroes_dict[hero_id]= hero_name

In [ ]:
def change_hero_name(row):
    if row in heroes_dict:
        return heroes_dict[row]
    
columns = [
    '1hero_id', 
    '2hero_id', 
    '3hero_id', 
    '4hero_id', 
    '5hero_id', 
    '6hero_id', 
    '7hero_id', 
    '8hero_id', 
    '9hero_id',
    '10hero_id',
    '11hero_id', 
    '12hero_id', 
    '13hero_id', 
    '14hero_id', 
    '15hero_id', 
    '16hero_id', 
    '17hero_id', 
    '18hero_id', 
    '19hero_id',
    '20hero_id'
    ]
    
for i in range(len(columns)):
    data[columns[i]] = data[columns[i]].apply(change_hero_name)

In [ ]:
data.head(5)

In [ ]:
np.max(data['duration'])
max_hours = (np.max(data['duration'])) / 60
max_hours

In [ ]:
def convert_to_minutes(row):
    return row / 60

data['minutes'] = data['duration'].apply(convert_to_minutes)

In [ ]:
groupby_minute = data.groupby(['minutes']).count()

In [ ]:
# groupby_minute.head()

In [ ]:
len(groupby_minute)

In [ ]:
groupby_minute.index

In [ ]:
max(groupby_minute.index)

In [ ]:
plt.hist(groupby_minute.index, histtype='bar', color='green')

In [ ]:
# # As there should be no bias, lets see which side of the map has the most win or is it equal

In [ ]:
base1 = data.loc[data['radiant_win'] == True]

In [ ]:
len(base1)

In [ ]:
base2 = data.loc[data['radiant_win'] == False]

In [ ]:
len(base2)

In [ ]:
total = len(base1) + len(base2)

In [ ]:
total

In [ ]:
len(data)

In [ ]:
temp_a = ['radiant', 'dire']
temp_b = [len(base1), len(base2)]

In [ ]:
plt.bar(temp_a, temp_b)

In [ ]:
# # Let's Analyze the Draft
# # As each draft must have it's own machine learning and send the data to the next machine learning model

In [ ]:
first_pick = data.loc[data['1is_pick'] == False]

In [ ]:
len(first_pick)

In [ ]:
first_picks = data.loc[data['1is_pick'] == True]

In [ ]:
len(first_picks)

In [ ]:
# # Checking the draft meta 
print(data.loc[0]['1is_pick'], data.loc[10]['1team_id'])
print(data.loc[0]['2is_pick'], data.loc[10]['2team_id'])
print(data.loc[0]['3is_pick'], data.loc[10]['3team_id'])
print(data.loc[0]['4is_pick'], data.loc[10]['4team_id'])
print(data.loc[0]['5is_pick'], data.loc[10]['5team_id'])
print(data.loc[0]['6is_pick'], data.loc[10]['6team_id'])
print(data.loc[0]['7is_pick'], data.loc[10]['7team_id'])
print(data.loc[0]['8is_pick'], data.loc[10]['8team_id'])
print(data.loc[0]['9is_pick'], data.loc[10]['9team_id'])
print(data.loc[0]['10is_pick'], data.loc[10]['10team_id'])
print(data.loc[0]['11is_pick'], data.loc[10]['11team_id'])
print(data.loc[0]['12is_pick'], data.loc[10]['12team_id'])
print(data.loc[0]['13is_pick'], data.loc[10]['13team_id'])
print(data.loc[0]['14is_pick'], data.loc[10]['14team_id'])
print(data.loc[0]['15is_pick'], data.loc[10]['15team_id'])
print(data.loc[0]['16is_pick'], data.loc[10]['16team_id'])
print(data.loc[0]['17is_pick'], data.loc[10]['17team_id'])
print(data.loc[0]['18is_pick'], data.loc[10]['18team_id'])
print(data.loc[0]['19is_pick'], data.loc[10]['19team_id'])
print(data.loc[0]['20is_pick'], data.loc[10]['20team_id'])

In [ ]:
# # Getting the ban for 1hero_id as 1is_pick is false
# # Let us supposr that 1st ban is from human, (it will be training data for us)

In [5]:
X_main = data['1hero_id']
y_main = data['2hero_id']

In [8]:
# # # Convert the string into label encoder
# encoder = LabelEncoder()

In [9]:
# X = encoder.fit_transform(X_main)
# y = encoder.transform(y_main)

In [13]:
X = np.array(X_main).reshape(-1, 1)
y = np.array(y_main).reshape(-1, 1)

In [16]:
model = LogisticRegression(n_jobs=4)

In [17]:
model.fit(X, y)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=4,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
model.predict(np.array([17, 23, 34, 84, 65]).reshape(-1, 1))

array([75, 75, 75, 75, 75])

In [20]:
# # Getting the 4th ban now using 1, 2 and 3

In [75]:
hero_1 = data['1hero_id']
hero_2 = data['2hero_id']
hero_3 = data['3hero_id']
hero_4 = data['4hero_id']
hero_5 = data['5hero_id']

In [76]:
test1 = pd.DataFrame(columns=['1hero_id', '2hero_id', '3hero_id', '4hero_id', '5hero_id'])

In [77]:
test1['1hero_id'] = hero_1
test1['2hero_id'] = hero_2
test1['3hero_id'] = hero_3
test1['4hero_id'] = hero_4
test1['5hero_id'] = hero_5

In [78]:
test1.head(2)

,1hero_id,2hero_id,3hero_id,4hero_id,5hero_id
0,73,69,71,7,28
1,69,102,32,61,74


In [79]:
X = test1.as_matrix()
X

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[ 73,  69,  71,   7,  28],
       [ 69, 102,  32,  61,  74],
       [ 85,  68,  75,   2,  73],
       ...,
       [ 44,  29,  12,  17,  11],
       [  1,  57,  50,  85,  29],
       [ 71, 100,   7,  85,  39]])

In [80]:
y = np.array(data['6hero_id'])

In [81]:
y

array([ 68., 112.,  50., ...,  18.,  28.,  11.])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
model.fit(X_train, y_train)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=4,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [84]:
y_pred = model.predict(X_test)

In [85]:
from sklearn.metrics import accuracy_score

In [86]:
accuracy_score(y_test, y_pred)

0.09412211256913566

In [88]:
np.unique(y_pred)

array([ 5.,  7., 30., 75.])

In [90]:
np.unique(y_test)

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,
        35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,
        46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,
        57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,
        68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,
        79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,
        90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.,
       101., 102., 103., 104., 105., 106., 110., 112., 113.])

In [ ]:
data['2hero_id']

In [ ]:
a = [20, 2, 3, 4, 1, 1, 1, 4, 1, 1]

In [ ]:
plt.hist(a)